In [3]:
import tensorflow as tf
gpus = tf.config.list_physical_devices('GPU')
tf.config.set_visible_devices(gpus[0], 'GPU')
tf.config.experimental.set_memory_growth(gpus[0], True)
# tf.config.set_logical_device_configuration(
# gpus[0],
# [tf.config.LogicalDeviceConfiguration(memory_limit=9216  )])

In [4]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

model_name = "PlanTL-GOB-ES/roberta-base-bne"
tokenizer = AutoTokenizer.from_pretrained(model_name)

model = TFAutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3,
                                                             from_pt=True)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFRobertaForSequenceClassification: ['roberta.embeddings.position_ids']
- This IS expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFRobertaForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFRobertaForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.dense.weight', 'classifier.dense.bias', 'classifier.out_proj.weight', 'classifier.out_proj.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predicti

In [6]:
from datasets import load_dataset, DatasetDict

raw_dataset = DatasetDict.load_from_disk(".././data")

In [7]:
raw_dataset

DatasetDict({
    train: Dataset({
        features: ['Title', 'Opinion', 'Polarity', 'Attraction', '__index_level_0__'],
        num_rows: 23944
    })
    test: Dataset({
        features: ['Title', 'Opinion', 'Polarity', 'Attraction', '__index_level_0__'],
        num_rows: 5987
    })
})

In [10]:
def tokenize(example):
    tokenized_example = tokenizer(example["Opinion"], truncation=True)
    tokenized_example["label"] = example["Attraction"]
    return tokenized_example


tokenized_dataset = raw_dataset.map(tokenize, batched=True, remove_columns=raw_dataset["train"].column_names)

  0%|          | 0/24 [00:00<?, ?ba/s]

  0%|          | 0/6 [00:00<?, ?ba/s]

In [11]:
tokenized_dataset["train"][0]

{'input_ids': [0,
  454,
  1936,
  369,
  7658,
  644,
  364,
  341,
  2762,
  313,
  533,
  7658,
  11062,
  68,
  859,
  7082,
  342,
  387,
  22319,
  644,
  28172,
  66,
  623,
  344,
  1416,
  1233,
  8883,
  11062,
  68,
  4742,
  2762,
  5209,
  341,
  5089,
  344,
  17984,
  467,
  1713,
  342,
  341,
  2090,
  27152,
  313,
  1488,
  55,
  2],
 'attention_mask': [1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1,
  1],
 'label': 0}

In [12]:
from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")

In [13]:
batch_size = 8

tf_train_dataset = tokenized_dataset["train"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "label"],
    collate_fn=data_collator,
    shuffle=True,
    batch_size=batch_size,
)

tf_eval_dataset = tokenized_dataset["test"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "label"],
    collate_fn=data_collator,
    shuffle=False,
    batch_size=batch_size,
)

In [14]:
model.config.num_labels

3

In [15]:
from tensorflow.keras import mixed_precision
policy = mixed_precision.Policy('mixed_float16')
mixed_precision.set_global_policy(policy)

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: GeForce RTX 3060, compute capability 8.6


In [16]:
from transformers import create_optimizer

num_epochs = 3
num_train_steps = len(tf_train_dataset) * num_epochs

optimizer, schedule = create_optimizer(
    init_lr=2e-5,
    num_warmup_steps=0,
    num_train_steps=num_train_steps,
    weight_decay_rate=0.01,
)
model.compile(optimizer=optimizer)

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! Please ensure your labels are passed as keys in the input dict so that they are accessible to the model during the forward pass. To disable this behaviour, please pass a loss argument, or explicitly pass loss=None if you do not want your model to compute a loss.


In [ ]:
from transformers.keras_callbacks import PushToHubCallback
callback = PushToHubCallback(output_dir="classificationEsp1_Attraction", tokenizer=tokenizer)

model.fit(
    tf_train_dataset,
    validation_data=tf_eval_dataset,
    callbacks=[callback],
    epochs=num_epochs
)

Cloning https://huggingface.co/javilonso/classificationEsp1_Attraction into local empty directory.


Epoch 1/3
2993/2993 [==============================] - 1113s 366ms/step - loss: 0.0913 - val_loss: 0.0654
Epoch 2/3
2993/2993 [==============================] - 1096s 366ms/step - loss: 0.0253 - val_loss: 0.0402
Epoch 3/3
2993/2993 [==============================] - 1096s 366ms/step - loss: 0.0070 - val_loss: 0.0398


In [ ]:
import numpy as np
from datasets import load_metric
 
load_accuracy = load_metric("accuracy")
load_f1 = load_metric("f1")
for batch in tf_eval_dataset:
    logits = model.predict(batch)["logits"]
    labels = batch["labels"]
    predictions = np.argmax(logits, axis=-1)
    load_accuracy.add_batch(predictions=predictions, references=labels)
    load_f1.add_batch(predictions=predictions, references=labels)


accuracy = load_accuracy.compute()["accuracy"]
f1 = load_f1.compute(average="None")["f1"]

In [ ]:
print(f'Accuracy: {accuracy:.2f}')
print(f'F1: {f1:.2f}')


In [3]:
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification

model_name = "classificationEsp1_Attraction"
tokenizer = AutoTokenizer.from_pretrained(model_name)

model = TFAutoModelForSequenceClassification.from_pretrained(model_name, num_labels=3)

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at classificationEsp1_Attraction.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.


In [4]:
model.push_to_hub("javilonso/classificationEsp1_Attraction")
tokenizer.push_to_hub("javilonso/classificationEsp1_Attraction")

/home/javilonso/.local/lib/python3.6/site-packages/huggingface_hub/hf_api.py:1004: FutureWarning: `create_repo` now takes `token` as an optional positional argument. Be sure to adapt your code!
  FutureWarning,
Cloning https://huggingface.co/javilonso/classificationEsp1_Attraction into local empty directory.


Download file tf_model.h5:   0%|          | 16.6k/476M [00:00<?, ?B/s]

Clean file tf_model.h5:   0%|          | 1.00k/476M [00:00<?, ?B/s]

Upload file tf_model.h5:   0%|          | 32.0k/476M [00:00<?, ?B/s]

To https://huggingface.co/javilonso/classificationEsp1_Attraction
   7fbde7c..5b5e70b  main -> main

To https://huggingface.co/javilonso/classificationEsp1_Attraction
   5b5e70b..9255f5a  main -> main



'https://huggingface.co/javilonso/classificationEsp1_Attraction/commit/9255f5a715ac71c1ae85f5ad0fc468dd65b45fb9'

In [ ]:
# Tests ------

check_point = "javilonso/classificationEsp1_Attraction"

from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
model = TFAutoModelForSequenceClassification.from_pretrained(check_point)
tokenizer = AutoTokenizer.from_pretrained(check_point)


from datasets import load_dataset, DatasetDict

raw_dataset = DatasetDict.load_from_disk(".././data")

def tokenize(example):
    tokenized_example = tokenizer(example["Opinion"], truncation=True)
    tokenized_example["label"] = example["Attraction"]
    return tokenized_example


tokenized_dataset = raw_dataset.map(tokenize, batched=True, remove_columns=raw_dataset["train"].column_names)


from transformers import DataCollatorWithPadding

data_collator = DataCollatorWithPadding(tokenizer=tokenizer, return_tensors="tf")


tf_train_dataset = tokenized_dataset["train"].to_tf_dataset(
    columns=["attenti on_mask", "input_ids", "label"],
    collate_fn=data_collator,
    shuffle=True,
    batch_size=16,
)

tf_eval_dataset = tokenized_dataset["test"].to_tf_dataset(
    columns=["attention_mask", "input_ids", "label"],
    collate_fn=data_collator,
    shuffle=False,
    batch_size=16,
)

import numpy as np
from datasets import load_metric
 
load_accuracy = load_metric("accuracy")
load_f1 = load_metric("f1")
for batch in tf_eval_dataset:
    logits = model.predict(batch)["logits"]
    labels = batch["labels"]
    predictions = np.argmax(logits, axis=-1)
    load_accuracy.add_batch(predictions=predictions, references=labels)
    load_f1.add_batch(predictions=predictions, references=labels)


accuracy = load_accuracy.compute()["accuracy"]
f1 = load_f1.compute(average=None)["f1"]

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

All the layers of TFRobertaForSequenceClassification were initialized from the model checkpoint at javilonso/classificationEsp1_Attraction.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFRobertaForSequenceClassification for predictions without further training.
Loading cached processed dataset at ../data/train/cache-1f28644c3516951f.arrow
Loading cached processed dataset at ../data/test/cache-82babf341eaded68.arrow


In [7]:
print(f'Accuracy: {accuracy:.4f}')
print(f'F1: {f1}')


Accuracy: 0.9883
F1: [0.99079755 0.97993411 0.99385343]
